In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [3]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 10.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 294kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.36MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.02MB/s]


In [4]:
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [5]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = Net().to(device)
print(model)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss() # difference of the measure between two probability distributions
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # lr will be the 1st parameter to tune when training

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.inference_mode(): # more modern and descriptive than torch.no_grad()
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    


In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308501  [    0/60000]
loss: 0.266020  [ 6400/60000]
loss: 0.183315  [12800/60000]
loss: 0.247883  [19200/60000]
loss: 0.139780  [25600/60000]
loss: 0.354818  [32000/60000]
loss: 0.119968  [38400/60000]
loss: 0.219135  [44800/60000]
loss: 0.294453  [51200/60000]
loss: 0.175370  [57600/60000]
Test Error: 
 Accuracy: 95.4%, Avg loss: 0.146124 

Epoch 2
-------------------------------
loss: 0.114065  [    0/60000]
loss: 0.089698  [ 6400/60000]
loss: 0.104708  [12800/60000]
loss: 0.095755  [19200/60000]
loss: 0.062179  [25600/60000]
loss: 0.143522  [32000/60000]
loss: 0.049149  [38400/60000]
loss: 0.132691  [44800/60000]
loss: 0.192176  [51200/60000]
loss: 0.099416  [57600/60000]
Test Error: 
 Accuracy: 97.0%, Avg loss: 0.095113 

Epoch 3
-------------------------------
loss: 0.064524  [    0/60000]
loss: 0.045703  [ 6400/60000]
loss: 0.039620  [12800/60000]
loss: 0.129522  [19200/60000]
loss: 0.051017  [25600/60000]
loss: 0.041989  [32000/600

In [9]:
torch.save(model.state_dict(), "mnist_base_model.pth") # save the weights
print("Saved PyTorch Model State to mnist_base_model.pth")

Saved PyTorch Model State to mnist_base_model.pth


In [13]:
model = Net().to(device)
model.load_state_dict(torch.load("mnist_base_model.pth"))
model.eval()

from PIL import Image

image_path = "number_5.png"
image = Image.open(image_path).convert("L") # convert to grayscale
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((28, 28)),
])
image = transform(image).to(device) # add batch dimension

with torch.inference_mode():
    output = model(image.unsqueeze(0)) # add batch dimension
    predicted_label = output.argmax(1).item()
    print(f"Predicted label: {predicted_label}")

Predicted label: 9
